In [1]:
import os
import json
import tqdm
import requests

from bs4 import BeautifulSoup

In [2]:
def get_url(limit, offset):
    return 'https://cnn-prod.content.agilesvcs.com/110/default/pt-BR/search?contentTypes=LNK&includeImages=Background&CA_ComponentType=Article&orderBy=changedAt%3AD&limit={}&offset={}&term=coronavirus'.format(limit, offset)

In [3]:
page = requests.get(get_url(limit=100, offset=0))

In [4]:
data = json.loads(page.content)

In [5]:
d = data['Content']['List'][0]

In [6]:
d['Title']

'Bruno Covas testa positivo para o novo coronavírus'

In [8]:
'https://www.cnnbrasil.com.br/' + d['Url']

'https://www.cnnbrasil.com.br/nacional/2020/06/13/bruno-covas-testa-positivo-para-novo-coronavirus'

In [9]:
' '.join([p.text for p in BeautifulSoup(d['Html'], 'html.parser').find_all('p')])

'O prefeito de São Paulo Bruno Covas (PSDB) foi diagnosticado com o novo coronavírus. De acordo com a assessoria de imprensa do município, a doença foi detectada num exame de rotina. Em vídeo nas redes sociais, o prefeito disse que não apresenta sintomas e que não irá se licenciar do cargo. Em tratamento contra um câncer, o tucano é considerado parte do grupo de risco ao novo coronavírus. "A Prefeitura de São Paulo informa que neste sábado (13/06), após teste preventivo de rotina, o prefeito Bruno Covas obteve diagnóstico positivo para coronavírus", diz a nota. "Ele passa bem, não apresenta sintomas e recebeu recomendação de seu médico, Dr. Davi Uip, para permanecer trabalhando em casa e em observação pelos próximos dias", conclui. O prefeito divulgou um vídeo em suas redes sociais em que comenta o diagnóstico: “Depois de quatro resultados negativos, hoje, infelizmente, eu testei positivo para a covid-19. O orientação do meu médico já que não tenho nenhum sintoma é ficar dentro de casa

In [10]:
articles = []
for offset in tqdm.tqdm(range(0, 1275, 100)):
    page = requests.get(get_url(limit=100, offset=offset))
    data = json.loads(page.content)
    for d in data['Content']['List']:
        articles.append(
            {
                'url': 'https://www.cnnbrasil.com.br/' + d['Url'],
                'title': d['Title'],
                'text': ' '.join([p.text for p in BeautifulSoup(d['Html'], 'html.parser').find_all('p')])
            }
        )

100%|██████████| 13/13 [00:16<00:00,  1.24s/it]


In [11]:
import pandas

In [13]:
df = pandas.DataFrame(articles)
df

,url,title,text
0,https://www.cnnbrasil.com.br/nacional/2020/06/...,Bruno Covas testa positivo para o novo coronav...,O prefeito de São Paulo Bruno Covas (PSDB) foi...
1,https://www.cnnbrasil.com.br/nacional/2020/06/...,Curitiba vai retomar medidas de restrição devi...,Jardim Botânico de Curitiba; praças e parques ...
2,https://www.cnnbrasil.com.br/tecnologia/2020/0...,Japão quer lançar aplicativo de vigilância con...,O Japão pretende lançar um aplicativo para sma...
3,https://www.cnnbrasil.com.br/internacional/202...,Viúva do médico que alertou sobre coronavírus ...,"""Você consegue ver do céu? O último presente q..."
4,https://www.cnnbrasil.com.br/esporte/2020/06/1...,NBA agenda testes de coronavírus para jogadore...,A National Basketball Association (NBA) estabe...
...,...,...,...
1270,https://www.cnnbrasil.com.br/saude/2020/02/27/...,Farmácias têm falta de máscaras após confirmaç...,Com a confirmação do primeiro caso de contamin...
1271,https://www.cnnbrasil.com.br/business/2020/02/...,Ibovespa tem nova queda com mercado ainda preo...,Preocupações com a propagação do novo coronaví...
1272,https://www.cnnbrasil.com.br/internacional/202...,Japonesa testa positivo pela segunda vez para ...,TÓQUIO - Uma guia de ônibus turístico no Japão...
1273,https://www.cnnbrasil.com.br/saude/2020/02/26/...,Primeiro brasileiro com coronavírus tem sintom...,O primeiro brasileiro com diagnóstico confir...


In [19]:
df.to_csv('data/cnn.csv', index=False, sep='|')